In [ ]:
!pip install tensorflow


In [98]:
!pip install xgboost

     --------------------------------------- 70.9/70.9 MB 16.0 MB/s eta 0:00:00


In [114]:
import pandas as pd
import numpy as np
import librosa
import librosa.display
import os
import sys
import seaborn as sb
import matplotlib.pyplot as plt
import IPython.display as ipd
import random
import multiprocessing as mp
from tqdm import tqdm
tqdm.pandas()



import warnings
warnings.simplefilter("ignore", UserWarning)

LOAD DATASET

In [3]:
#load data and explore audio sample
ravdess_path = 'C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24'
crema_path = 'C:/Users/sarim/Downloads/CREMA/AudioWAV'
tess_path= 'C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data'


RAVDESS

In [4]:
#create a list of the audio directories in ravdess_path
ravdess_dir_list = os.listdir(ravdess_path)
ravdess_dir_list.sort()

#extract emotion label for each audio in the created directory
audio_emotion = []
audio_path = []
for dir in ravdess_dir_list:
    actor = os.listdir(ravdess_path + '/' + dir)#iterate and extract audio files for each actor
    for audio_file in actor:
        part = audio_file.split('.')[0].split('-')
        audio_emotion.append(int(part[2]))#The third index part in each audio reps the emotion label
        audio_path.append(ravdess_path + '/' + dir + '/' + audio_file)

#Insert audio_path and emotions into a DataFrame and replace numerical emotion labels with actual emotions
ravdess_emotion_df = pd.DataFrame(audio_emotion, columns=['Emotion'])
ravdess_emotion_df = ravdess_emotion_df.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'})
audio_path_df = pd.DataFrame(audio_path, columns=['Path'])

Ravdess_df = pd.concat([ravdess_emotion_df, audio_path_df], axis = 1)
Ravdess_df.Emotion.value_counts(), Ravdess_df.Path.shape

(calm        192
 happy       192
 sad         192
 angry       192
 fear        192
 disgust     192
 surprise    192
 neutral      96
 Name: Emotion, dtype: int64,
 (1440,))

In [5]:
#Lets view the full dataset
pd.set_option('display.max_colwidth', -1)

Ravdess_df.sample(10)

C:\Users\sarim\AppData\Local\Temp\ipykernel_25924\260657533.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Emotion,Path
781,neutral,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_14/03-01-01-01-01-02-14.wav
784,calm,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_14/03-01-02-01-01-01-14.wav
689,angry,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_12/03-01-05-01-01-02-12.wav
147,sad,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_03/03-01-04-02-02-02-03.wav
655,surprise,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_11/03-01-08-01-02-02-11.wav
292,surprise,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_05/03-01-08-01-01-01-05.wav
1032,happy,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_18/03-01-03-01-01-01-18.wav
948,disgust,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_16/03-01-07-02-01-01-16.wav
876,fear,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_15/03-01-06-01-01-01-15.wav
702,fear,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_12/03-01-06-02-02-01-12.wav


CREMA-D

In [6]:
#create a list of the audio directories in ravdess_path
crema_dir_list = os.listdir(crema_path)
crema_dir_list.sort()

#extract emotion label for each audio in the created directory

audio_emotion = []
audio_path = []

for wav in crema_dir_list:
    audio_path.append(crema_path + '/' + wav)
    part=wav.split('_')
    if part[2] == 'SAD':
        audio_emotion.append('sad')
    elif part[2] == 'ANG':
        audio_emotion.append('angry')
    elif part[2] == 'DIS':
        audio_emotion.append('disgust')
    elif part[2] == 'FEA':
        audio_emotion.append('fear')
    elif part[2] == 'HAP':
        audio_emotion.append('happy')
    elif part[2] == 'NEU':
        audio_emotion.append('neutral')
    else:
        audio_emotion.append('Unknown')

# dataframe for emotion of files
crema_emotion_df = pd.DataFrame(audio_emotion, columns=['Emotion'])

# dataframe for path of files.
audio_path_df = pd.DataFrame(audio_path, columns=['Path'])
Crema_D_df = pd.concat([crema_emotion_df, audio_path_df], axis=1)

Crema_D_df.Emotion.value_counts(), Crema_D_df.Path.shape


(angry      1271
 disgust    1271
 fear       1271
 happy      1271
 sad        1271
 neutral    1087
 Name: Emotion, dtype: int64,
 (7442,))

In [7]:
#Lets view the full dataset
pd.set_option('display.max_colwidth', -1)

Crema_D_df.sample(10)

C:\Users\sarim\AppData\Local\Temp\ipykernel_25924\1053867059.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Emotion,Path
491,angry,C:/Users/sarim/Downloads/CREMA/AudioWAV/1007_DFA_ANG_XX.wav
6357,sad,C:/Users/sarim/Downloads/CREMA/AudioWAV/1078_TAI_SAD_XX.wav
3256,disgust,C:/Users/sarim/Downloads/CREMA/AudioWAV/1040_WSI_DIS_XX.wav
1042,disgust,C:/Users/sarim/Downloads/CREMA/AudioWAV/1013_TSI_DIS_XX.wav
5460,happy,C:/Users/sarim/Downloads/CREMA/AudioWAV/1067_TIE_HAP_XX.wav
4689,angry,C:/Users/sarim/Downloads/CREMA/AudioWAV/1058_ITS_ANG_XX.wav
6255,happy,C:/Users/sarim/Downloads/CREMA/AudioWAV/1077_IWL_HAP_XX.wav
2536,fear,C:/Users/sarim/Downloads/CREMA/AudioWAV/1032_IEO_FEA_LO.wav
2431,fear,C:/Users/sarim/Downloads/CREMA/AudioWAV/1030_TSI_FEA_XX.wav
6111,neutral,C:/Users/sarim/Downloads/CREMA/AudioWAV/1075_TAI_NEU_XX.wav


TESS

In [8]:
tess_dir_list = os.listdir(tess_path)
tess_dir_list.sort()

audio_emotion = []
audio_path = []

for dir in tess_dir_list:
    folders = os.listdir(tess_path + '/' + dir)
    for wav in folders:
        part = wav.split('.')[0]
        part = part.split('_')[2]
        if part=='ps':
            audio_emotion.append('surprise')
        else:
            audio_emotion.append(part)
        audio_path.append(tess_path + '/' + dir + '/' + wav)

tess_emotion_df = pd.DataFrame(audio_emotion, columns=['Emotion'])

audio_path_df = pd.DataFrame(audio_path, columns=['Path'])
Tess_df = pd.concat([tess_emotion_df, audio_path_df], axis=1)
Tess_df.Emotion.value_counts(), Tess_df.Path.shape

(fear        400
 surprise    400
 sad         400
 angry       400
 disgust     400
 happy       400
 neutral     400
 Name: Emotion, dtype: int64,
 (2800,))

In [9]:
#Lets view the full dataset
pd.set_option('display.max_colwidth', -1)

Tess_df.head()

C:\Users\sarim\AppData\Local\Temp\ipykernel_25924\4236753067.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


,Emotion,Path
0,fear,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_back_fear.wav
1,fear,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_bar_fear.wav
2,fear,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_base_fear.wav
3,fear,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_bath_fear.wav
4,fear,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/OAF_Fear/OAF_bean_fear.wav


DATA CONCATENATION

In [10]:
main_audio_df = pd.concat([Ravdess_df, Crema_D_df, Tess_df], axis=0,ignore_index=True) #concat all df except savee
main_audio_df

,Emotion,Path
0,neutral,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-01-01.wav
1,neutral,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-01-02-01.wav
2,neutral,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-01-01.wav
3,neutral,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-01-01-02-02-01.wav
4,calm,C:/Users/sarim/Downloads/RAVDESS/audio_speech_actors_01-24/Actor_01/03-01-02-01-01-01-01.wav
...,...,...
11677,sad,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/YAF_sad/YAF_witch_sad.wav
11678,sad,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/YAF_sad/YAF_yearn_sad.wav
11679,sad,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/YAF_sad/YAF_yes_sad.wav
11680,sad,C:/Users/sarim/Downloads/TESS/TESS Toronto emotional speech set data/TESS Toronto emotional speech set data/YAF_sad/YAF_young_sad.wav


SIGNAL PROCESSING

In [11]:
working_df = main_audio_df.copy()
working_df["Signal"] = None
working_df["Sampling Rate"] = None

for row, path in tqdm(enumerate(working_df['Path']), total=len(working_df)):
    signal, rate = librosa.load(path, duration=2.5, offset=0.6)
    working_df.at[row, "Signal"] = signal
    working_df.at[row, "Sampling Rate"] = rate

  0%|          | 0/11682 [00:00<?, ?it/s]

100%|██████████| 11682/11682 [00:37<00:00, 307.58it/s]


In [ ]:
working_df

DATA AUGMENTATION

In [13]:
#Performing Noise addition, time and pitch shifting
def noise(signal):
    noisy_audio = signal+0.035*np.random.uniform()*np.amax(signal)*np.random.normal(size=signal.shape[0])
    return noisy_audio

def pitch(signal, sample_rate, n_steps=0.5):
    shifted_audio = librosa.effects.pitch_shift(signal, sr=sample_rate, n_steps=0.5)
    return shifted_audio

def time_stretch(signal, rate=1.2):
    stretched_audio = librosa.effects.time_stretch(signal, rate=1.2)
    return  stretched_audio

In [33]:
#create DFs for the each augumentation technique
noise_df = working_df.copy()
pitch_stretch_df = working_df.copy()
time_stretch_df = working_df.copy()

In [15]:
noise_df["Signal"] = working_df["Signal"].progress_apply(noise)

100%|██████████| 11682/11682 [00:15<00:00, 742.48it/s]


In [17]:
#apply time_stretch to audio signal
time_stretch_df["Signal"] = working_df["Signal"].progress_apply(time_stretch)

  0%|          | 0/11682 [00:00<?, ?it/s]

100%|██████████| 11682/11682 [17:31<00:00, 11.11it/s]


In [37]:
pitch_stretch_df["Signal"] = time_stretch_df.progress_apply(lambda x : pitch(x['Signal'], 22050), axis = 1)

  0%|          | 0/11682 [00:00<?, ?it/s]

100%|██████████| 11682/11682 [26:03<00:00,  7.47it/s]


In [56]:
#Concat all augmented audio

augmented_df = pd.concat([working_df, noise_df, pitch_stretch_df], axis=0,ignore_index=True)
augmented_df.shape

(35046, 4)

FEATURE EXTRACTION

In [39]:
def extract_features(audio, sample_rate):
   
    try:
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        chroma = librosa.feature.chroma_stft(y=audio, sr=sample_rate)
        spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sample_rate)
        tonnetz = librosa.feature.tonnetz(y=audio, sr=sample_rate)
        zcr = librosa.feature.zero_crossing_rate(y=audio)
        
    except Exception as e:
        print("Error encountered while parsing file: ", e)
        return None 
     
    return {"mfccs_mean":np.mean(mfccs,axis=1), "mfccs_std":np.std(mfccs,axis=1),
            "chroma_mean":np.mean(chroma,axis=1), "chroma_std":np.std(chroma,axis=1),
            "spectral_contrast_mean":np.mean(spectral_contrast,axis=1), "spectral_contrast_std":np.std(spectral_contrast,axis=1),
            "tonnetz_mean":np.mean(tonnetz,axis=1), "tonnetz_std":np.std(tonnetz,axis=1),
            "zcr_mean":np.mean(zcr,axis=1), "zcr_std":np.std(zcr,axis=1)}


In [40]:
features_df = augmented_df.copy()
features_df = pd.concat([features_df.drop(['Signal'], axis=1), features_df['Signal'].progress_apply(lambda x: pd.Series(extract_features(x, sample_rate=22050)))], axis=1)


  0%|          | 0/35046 [00:00<?, ?it/s]

100%|██████████| 35046/35046 [3:10:34<00:00,  3.06it/s]  


In [ ]:
features_df.head()

In [44]:

# saving the dataframe
features_df.to_csv('Extracted_features.csv')
features_df.to_pickle('Extracted_features.pkl')


In [ ]:
def compute_mel_spectrogram_features(signal, sr):
    mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr).T
    return np.mean(mel_spectrogram, axis=0), np.std(mel_spectrogram, axis=0)

mel_features_df = augmented_df.copy()
mel_features_df[['mel_spectrogram_mean', 'mel_spectrogram_std']] = mel_features_df['Signal'].progress_apply(
    lambda x: pd.Series(compute_mel_spectrogram_features(x, sr=22050))
)


mel_features_df.head()

In [ ]:
# Now, let's concatenate the mel features to the features_df
updated_features_df = pd.concat([features_df, mel_features_df[['mel_spectrogram_mean', 'mel_spectrogram_std']]], axis=1)

updated_features_df.head()

In [69]:
#save updated features
updated_features_df.to_csv('Extracted_features(update).csv')
updated_features_df.to_pickle('Extracted_features(update).pkl')

DATA PREPROCESSING

In [63]:
#Prep features for feature selection
# Create new dataframe to hold flattened features
flattened_features_df = updated_features_df.copy()

# Define function to flatten a feature
def flatten_feature(df, column):
    flat_df = pd.DataFrame(df[column].to_list(), index=df.index)
    flat_df.columns = [f'{column}_{i+1}' for i in range(flat_df.shape[1])]
    return df.drop(column, axis=1).join(flat_df)

# List of columns to flatten
columns_to_flatten = ['mfccs_mean', 'mfccs_std', 'chroma_mean', 'chroma_std',
                      'spectral_contrast_mean', 'spectral_contrast_std',
                      'tonnetz_mean', 'tonnetz_std', 'zcr_mean', 'zcr_std',
                      'mel_spectrogram_mean', 'mel_spectrogram_std']

# Flatten each column
for column in columns_to_flatten:
    flattened_features_df = flatten_feature(flattened_features_df, column)


In [64]:
flattened_features_df.shape

(35046, 391)

In [71]:
#save flattened df
flattened_features_df.to_csv('Flattened_features.csv')
flattened_features_df.to_pickle('Flattened_features.pkl')

In [65]:
# DROP PATH COLUMN FOR MODELING
model_df = flattened_features_df.copy()
model_df.drop(columns=['Path','Sampling Rate'],inplace=True)

model_df.head()

,Emotion,mfccs_mean_1,mfccs_mean_2,mfccs_mean_3,mfccs_mean_4,mfccs_mean_5,mfccs_mean_6,mfccs_mean_7,mfccs_mean_8,mfccs_mean_9,...,mel_spectrogram_std_119,mel_spectrogram_std_120,mel_spectrogram_std_121,mel_spectrogram_std_122,mel_spectrogram_std_123,mel_spectrogram_std_124,mel_spectrogram_std_125,mel_spectrogram_std_126,mel_spectrogram_std_127,mel_spectrogram_std_128
0,neutral,-645.880737,72.619637,0.841305,16.399446,10.241591,0.653901,-4.328002,-4.258932,-14.575824,...,0.000013,0.000014,0.000009,0.000005,0.000006,0.000017,0.000028,0.000021,0.000019,0.000002
1,neutral,-634.663269,72.324066,-2.998578,20.209740,10.674217,-1.151011,-2.813592,-7.923956,-16.231958,...,0.000031,0.000022,0.000019,0.000024,0.000025,0.000051,0.000036,0.000045,0.000023,0.000001
2,neutral,-643.285645,74.262268,-1.228256,16.242317,5.608830,0.250306,-4.034254,-8.218330,-14.603807,...,0.000049,0.000050,0.000045,0.000064,0.000094,0.000130,0.000248,0.000141,0.000069,0.000007
3,neutral,-644.624451,69.160400,3.003006,16.580229,7.867353,2.587296,-2.231727,-8.238157,-12.929442,...,0.000065,0.000080,0.000069,0.000095,0.000453,0.000181,0.000155,0.000175,0.000102,0.000007
4,calm,-652.413757,87.086311,3.265245,20.345898,10.620335,1.831458,-5.369885,-6.077728,-14.500923,...,0.000034,0.000028,0.000022,0.000072,0.000030,0.000015,0.000018,0.000019,0.000015,0.000001


In [72]:
model_df.shape

(35046, 389)

PREPARING DATA FOR MODELLING

In [80]:
# Creating X(predictors) and Y(target)
X = model_df.drop(columns=['Emotion'])
y = model_df['Emotion']


In [ ]:
# Label encoding the target variable for traditional ML models
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encoding the target variable for deep learning models
y_onehot = to_categorical(y_encoded)

label_encoder.classes_

In [ ]:
# Train-validation-test split
X_temp, X_test, y_temp, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=10)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=10)

X_temp_dl, X_test_dl, y_temp_dl, y_test_dl = train_test_split(X, y_onehot, test_size=0.2, random_state=42)
X_train_dl, X_val_dl, y_train_dl, y_val_dl = train_test_split(X_temp_dl, y_temp_dl, test_size=0.2, random_state=42)


X_train.shape, X_train_dl.shape

In [3]:
# Scaling the features for traditional models
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# Scaling the features for deep learning models
X_train_dl = scaler.fit_transform(X_train_dl)
X_test_dl = scaler.transform(X_test_dl)
X_val_dl = scaler.transform(X_val_dl)


NameError: name 'StandardScaler' is not defined

In [132]:
# Creating a pipeline for preprocessing the deep learning data
# This step is necessary because the input to a CNN must be a 3D array (samples, timesteps, features)
# Reshaping the data to be suitable for CNN
X_train_dl = X_train_dl.reshape(X_train_dl.shape[0], X_train_dl.shape[1], 1)
X_test_dl = X_test_dl.reshape(X_test_dl.shape[0], X_test_dl.shape[1], 1)
X_val_dl = X_val_dl.reshape(X_val_dl.shape[0], X_val_dl.shape[1], 1)

X_train_dl.shape, X_test_dl.shape, X_val_dl.shape

((22428, 388, 1), (7010, 388, 1), (5608, 388, 1))

In [133]:
print(y_train_dl[0])
print(y_test_dl[0])
y_train_dl.shape, y_test_dl.shape


[0. 0. 1. 0. 0. 0. 0. 0.]
[0. 0. 1. 0. 0. 0. 0. 0.]


((22428, 8), (7010, 8))

BASELINES

The purpose of a baseline is to set a reasonably low bar to improve upon and for the purpose of this project to see if there might be a need
for feature selection. 
We'll be using Gaussian Naive Bayes and Decision Trees as baselines since we are dealing with continuous audio features extracted from sound clips

In [95]:
#BASELINES
# Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Predictions
y_pred_gnb = gnb.predict(X_test)

# Evaluate the model
print('--- Gaussian Naive Bayes ---')
print('Accuracy:', accuracy_score(y_test, y_pred_gnb))
print(classification_report(y_test, y_pred_gnb))



--- Gaussian Naive Bayes ---
Accuracy: 0.269472182596291
              precision    recall  f1-score   support

           0       0.66      0.27      0.38      1103
           1       0.15      0.93      0.25       123
           2       0.15      0.11      0.13      1082
           3       0.43      0.08      0.14      1113
           4       0.30      0.14      0.19      1101
           5       0.25      0.84      0.38       973
           6       0.27      0.10      0.14      1170
           7       0.30      0.54      0.39       345

    accuracy                           0.27      7010
   macro avg       0.31      0.38      0.25      7010
weighted avg       0.34      0.27      0.23      7010



In [96]:
# Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Predictions
y_pred_dt = dt.predict(X_test)

# Evaluate the model
print('\n--- Decision Tree ---')
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_dt))


--- Decision Tree ---
Accuracy: 0.7601997146932953
              precision    recall  f1-score   support

           0       0.84      0.82      0.83      1103
           1       0.79      0.76      0.78       123
           2       0.72      0.71      0.71      1082
           3       0.74      0.73      0.73      1113
           4       0.73      0.75      0.74      1101
           5       0.74      0.73      0.73       973
           6       0.76      0.80      0.78      1170
           7       0.84      0.83      0.83       345

    accuracy                           0.76      7010
   macro avg       0.77      0.77      0.77      7010
weighted avg       0.76      0.76      0.76      7010



The results from the Decision Tree classifier are quite promising with an accuracy of ~76%. It looks like the features are already capturing a good amount of the underlying structure of the data.

Gaussian Naive Bayes did not perform as well, which isn't surprising. Naive Bayes makes a strong assumption about the independence of features, which might not hold in this case.

MODELLING

Traditional Models

Support Vector Machine (SVM):

In [100]:
# Instantiate the model
svm = SVC()

# Fit on the training data
svm.fit(X_train, y_train)

# Predict on the test data
svm_preds = svm.predict(X_test)

# Generate and print the classification report
svm_report = classification_report(y_test, svm_preds)
print('SVM:', svm_report)


SVM:               precision    recall  f1-score   support

           0       0.71      0.72      0.71      1103
           1       0.34      0.67      0.45       123
           2       0.43      0.53      0.48      1082
           3       0.70      0.37      0.49      1113
           4       0.55      0.52      0.54      1101
           5       0.54      0.43      0.48       973
           6       0.52      0.71      0.60      1170
           7       0.78      0.79      0.79       345

    accuracy                           0.56      7010
   macro avg       0.57      0.59      0.57      7010
weighted avg       0.58      0.56      0.56      7010



K-Nearest Neighbors (KNN):

In [101]:
# Instantiate the model
knn = KNeighborsClassifier()

# Fit on the training data
knn.fit(X_train, y_train)

# Predict on the test data
knn_preds = knn.predict(X_test)

# Generate and print the classification report
knn_report = classification_report(y_test, knn_preds)
print('KNN:', knn_report)


KNN:               precision    recall  f1-score   support

           0       0.69      0.76      0.72      1103
           1       0.82      0.89      0.85       123
           2       0.50      0.63      0.56      1082
           3       0.58      0.54      0.56      1113
           4       0.66      0.56      0.61      1101
           5       0.63      0.57      0.59       973
           6       0.72      0.68      0.70      1170
           7       0.94      0.93      0.94       345

    accuracy                           0.64      7010
   macro avg       0.69      0.69      0.69      7010
weighted avg       0.65      0.64      0.64      7010



Random Forest:

In [102]:
# Instantiate the model
rf = RandomForestClassifier()

# Fit on the training data
rf.fit(X_train, y_train)

# Predict on the test data
rf_preds = rf.predict(X_test)

# Generate and print the classification report
rf_report = classification_report(y_test, rf_preds)
print('Random Forest:', rf_report)


Random Forest:               precision    recall  f1-score   support

           0       0.89      0.88      0.89      1103
           1       0.80      0.89      0.84       123
           2       0.80      0.80      0.80      1082
           3       0.91      0.75      0.82      1113
           4       0.83      0.84      0.84      1101
           5       0.79      0.82      0.81       973
           6       0.77      0.88      0.82      1170
           7       0.95      0.95      0.95       345

    accuracy                           0.84      7010
   macro avg       0.84      0.85      0.85      7010
weighted avg       0.84      0.84      0.84      7010



XGBoost:

In [103]:
# Instantiate the model
xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Fit on the training data
xgb.fit(X_train, y_train)

# Predict on the test data
xgb_preds = xgb.predict(X_test)

# Generate and print the classification report
xgb_report = classification_report(y_test, xgb_preds)
print('XGBoost:', xgb_report)


XGBoost:               precision    recall  f1-score   support

           0       0.92      0.91      0.91      1103
           1       0.94      0.95      0.95       123
           2       0.82      0.84      0.83      1082
           3       0.88      0.81      0.84      1113
           4       0.86      0.86      0.86      1101
           5       0.83      0.84      0.83       973
           6       0.84      0.89      0.86      1170
           7       0.98      0.96      0.97       345

    accuracy                           0.86      7010
   macro avg       0.88      0.88      0.88      7010
weighted avg       0.87      0.86      0.86      7010



Logistic Regression:

In [104]:
# Instantiate the model
lr = LogisticRegression(max_iter=1000)

# Fit on the training data
lr.fit(X_train, y_train)

# Predict on the test data
lr_preds = lr.predict(X_test)

# Generate and print the classification report
lr_report = classification_report(y_test, lr_preds)
print('Logistic Regression:', lr_report)


Logistic Regression:               precision    recall  f1-score   support

           0       0.73      0.71      0.72      1103
           1       0.69      0.77      0.73       123
           2       0.50      0.57      0.53      1082
           3       0.60      0.50      0.55      1113
           4       0.57      0.54      0.55      1101
           5       0.59      0.55      0.57       973
           6       0.59      0.68      0.63      1170
           7       0.77      0.80      0.78       345

    accuracy                           0.60      7010
   macro avg       0.63      0.64      0.63      7010
weighted avg       0.61      0.60      0.60      7010



VALIDATION

This will give a sense of how the model's performance varies across different subsets of the validation set. High variance across subsets can indicate a high-variance (overfitting) model, while consistent but poor performance can indicate a high-bias (underfitting) model.

In [106]:
# Calculate cross-validation accuracy for each model
svm_scores = cross_val_score(svm, X, y, cv=5)
knn_scores = cross_val_score(knn, X, y, cv=5)
rf_scores = cross_val_score(rf, X, y, cv=5)
xgb_scores = cross_val_score(xgb, X, y_encoded, cv=5)
lr_scores = cross_val_score(lr, X, y, cv=5)

# Print mean cross-validation accuracy for each model
print('SVM mean cross-validation accuracy:', np.mean(svm_scores))
print('KNN mean cross-validation accuracy:', np.mean(knn_scores))
print('Random Forest mean cross-validation accuracy:', np.mean(rf_scores))
print('XGBoost mean cross-validation accuracy:', np.mean(xgb_scores))
print('Logistic Regression mean cross-validation accuracy:', np.mean(lr_scores))



SVM mean cross-validation accuracy: 0.5557580115559595
KNN mean cross-validation accuracy: 0.7032190525773975
Random Forest mean cross-validation accuracy: 0.8829790391770598
XGBoost mean cross-validation accuracy: 0.8981024560026654
Logistic Regression mean cross-validation accuracy: 0.6018683661052051


Here, none of the models seem to be severely overfitting. The SVM model might be underfitting, though, since it has the lowest performance metrics. One common sign of overfitting is when your model performs well on the training set (high precision and recall) but poorly during cross-validation (low mean accuracy), but that doesn't seem to be the case here.

The Random Forest and XGBoost models seem to be the best performers on this task based on the provided metrics. They both have high f1-scores and cross-validation accuracies, suggesting that they are generalizing well to unseen data.

Simple CNN Model

In [121]:
# Number of output classes
num_classes = y_train_dl.shape[1] 

# Define model
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=5, activation='relu', input_shape=(X_train_dl.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
history = model.fit(X_train_dl, y_train_dl, epochs=20, validation_data=(X_test_dl, y_test_dl))


Epoch 1/20
877/877 [==============================] - 203s 231ms/step - loss: 1.6615 - accuracy: 0.3687 - val_loss: 1.3371 - val_accuracy: 0.4976
Epoch 2/20
877/877 [==============================] - 177s 202ms/step - loss: 1.3561 - accuracy: 0.4563 - val_loss: 1.1934 - val_accuracy: 0.5415
Epoch 3/20
877/877 [==============================] - 184s 210ms/step - loss: 1.2725 - accuracy: 0.4903 - val_loss: 1.1424 - val_accuracy: 0.5398
Epoch 4/20
877/877 [==============================] - 182s 207ms/step - loss: 1.2271 - accuracy: 0.5092 - val_loss: 1.1247 - val_accuracy: 0.5525
Epoch 5/20
877/877 [==============================] - 194s 221ms/step - loss: 1.1966 - accuracy: 0.5166 - val_loss: 1.1071 - val_accuracy: 0.5588
Epoch 6/20
877/877 [==============================] - 213s 243ms/step - loss: 1.1755 - accuracy: 0.5292 - val_loss: 1.0969 - val_accuracy: 0.5542
Epoch 7/20
877/877 [==============================] - 226s 257ms/step - loss: 1.1588 - accuracy: 0.5347 - val_loss: 1.0888 -

HYPERPARAMETER TUNING

POST-MODEL ANALYSIS

In [ ]:
# PRINT LOSS AND ACCURACY PERCENTAGE ON TEST SET

In [ ]:
#ACTUAL VS PREDICTED VALUES